# SMAI Assignment - 2

## Question - `3` : Multinomial Naïve Bayes

| | |
|- | -|
| Course | Statistical Methods in AI |
| Release Date | `16.02.2023` |
| Due Date | `24.02.2023` |

This question will have you working and experimenting with the Multinomial Naïve Bayes classifier. Initially, you will transform the given data in csv file to count matrix, then calculate the priors. Use those priors to compute likelyhoods according to Multinomial Naive Bayes and then classify the test data. Please note that use of `sklearn` implementations is only for the final question of the assignment, for other doubts regarding libraries you can reach out to the TAs.

The dataset is about `Spam SMS`. There is 1 attribute that is the `message`, and the class label which could be `spam` or `ham`. The data is present in `spam.csv`. It contains about 5-6000 samples.
For your convinience the data is already pre-processed and loaded, but I suggest you to just take a look at the code for your own knowledge, and parts vectorization is left up to you which could be easily done with the help of the given example code.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Reading text-based data using pandas

In [2]:
# read file into pandas using a relative path

df = pd.read_csv("./spam.csv", encoding='latin-1')
df.dropna(how="any", inplace=True, axis=1)
df.columns = ['label', 'message']

df.describe()

,label,message
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


## Pre-processing

- Our main issue with our data is that it is all in text format (strings). The classification algorithms that we usally use need some sort of numerical feature vector in order to perform the classification task. There are actually many methods to convert a corpus to a vector format. The simplest is the bag-of-words approach, where each unique word in a text will be represented by one number.

- As a first step, let's write a function that will split a message into its individual words and return a list. We'll also remove very common words, ('the', 'a', etc..). To do this we will take advantage of the NLTK library. It's pretty much the standard library in Python for processing text and has a lot of useful features. We'll only use some of the basic ones here.

In [3]:
import string
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    STOPWORDS = stopwords.words('english') + ['u', 'ü', 'ur', '4', '2', 'im', 'dont', 'doin', 'ure']
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return ' '.join([word for word in nopunc.split() if word.lower() not in STOPWORDS])

[nltk_data] Downloading package stopwords to /home/user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
df['message'] = df.message.apply(text_process)
df.head()

,label,message
0,ham,Go jurong point crazy Available bugis n great ...
1,ham,Ok lar Joking wif oni
2,spam,Free entry wkly comp win FA Cup final tkts 21s...
3,ham,dun say early hor c already say
4,ham,Nah think goes usf lives around though


In [5]:
df['label'] = df.label.map({'ham':0, 'spam':1})
df.head()

,label,message
0,0,Go jurong point crazy Available bugis n great ...
1,0,Ok lar Joking wif oni
2,1,Free entry wkly comp win FA Cup final tkts 21s...
3,0,dun say early hor c already say
4,0,Nah think goes usf lives around though


## Splitting the data

In [6]:
# split X and y into training and testing sets 
from sklearn.model_selection import train_test_split

X = df.message
y = df.label

print(f'X: {X.shape}')
print(f'y: {y.shape}')
print()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

print(f'X_train: {X_train.shape}')
print(f'y_train: {y_train.shape}')
print()

print(f'X_test: {X_test.shape}')
print(f'y_test: {y_test.shape}')


X: (5572,)
y: (5572,)

X_train: (4179,)
y_train: (4179,)

X_test: (1393,)
y_test: (1393,)


## Helper code / Example code for Representing text as Numerical data using Sci-kit learn

📌 From the scikit-learn documentation:
- Text Analysis is a major application field for machine learning algorithms. However the raw data, a sequence of symbols cannot be fed directly to the algorithms themselves as most of them expect numerical feature vectors with a fixed size rather than the raw text documents with variable length.
- We will use CountVectorizer to "convert text into a matrix of token counts":

In [7]:
# example text for model training (SMS messages)
simple_train = ['call you tonight', 'Call me a cab', 'Please call me... PLEASE!']

In [8]:
# import and instantiate CountVectorizer (with the default parameters)
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer()
simple_train = vect.fit_transform(simple_train)

vect.get_feature_names_out()

array(['cab', 'call', 'me', 'please', 'tonight', 'you'], dtype=object)

In [9]:
# convert sparse matrix to a dense matrix
simple_train.toarray()

array([[0, 1, 0, 0, 1, 1],
       [1, 1, 1, 0, 0, 0],
       [0, 1, 1, 2, 0, 0]])

In this scheme, features and samples are defined as follows:

- Each individual token occurrence frequency (normalized or not) is treated as a feature.
- The vector of all the token frequencies for a given document is considered a multivariate sample.

A corpus of documents can thus be represented by a matrix with one row per document and one column per token (e.g. word) occurring in the corpus.

In [10]:
# examine the vocabulary and document-term matrix together
pd.DataFrame(simple_train.toarray(), columns=vect.get_feature_names_out())

,cab,call,me,please,tonight,you
0,0,1,0,0,1,1
1,1,1,1,0,0,0
2,0,1,1,2,0,0


### Transform Testing data into a document-term matrix (using existing / training vocabulary)

- You are supposed to use the training vocabolary to make the count matrix for test data

In [11]:
simple_test = ["please don't call me"]

In [12]:
simple_test_dtm = vect.transform(simple_test)
simple_test_dtm.toarray()

array([[0, 1, 1, 1, 0, 0]])

In [13]:
# examine the vocabulary and document-term matrix together
pd.DataFrame(simple_test_dtm.toarray(), columns=vect.get_feature_names_out())

,cab,call,me,please,tonight,you
0,0,1,1,1,0,0


## Multinomial Naive Bayes Implementation

- Your task is to implement Mutlinomial Naive Bayes from scratch, you can use numpy to vectorize your code and matplotlib  to show your analysis.
- Below some information has given from the documentation about Multinomial Naive Bayes, this will give you some idea about using *Smoothing Priors*.
- There is a sub-question for experimenting with $\alpha > 0$, you don't have to implement it separetely, try to incomporate it in same Model Class / Function.

📌 From the scikit-learn documentation:

- Multinomial Naive Bayes implements the naive Bayes algorithm for multinomially distributed data, and is one of the two classic naive Bayes variants used in text classification (where the data are typically represented as word vector counts, although tf-idf vectors are also known to work well in practice).

- The distribution $\theta_y = (\theta_{y1}, \theta_{y2}, \dots, \theta_{yn})$ is parametrized by vectors for each class $y$, where $n$ is the number of features (in text classification, the size of the vocabulary) and $\theta_{yi}$ is the probability $P(x_i|y)$ of feature appearing in a sample belonging to class.

- The parameters $\theta_y$ is estimated by a smoothed version of maximum likelihood, i.e. relative frequency counting:

$$
\hat{\theta}_{yi} = \frac{N_{yi} + \alpha}{N_{y} + \alpha n}
$$

 where $N_{yi} = \sum_{x \in T}{x_i}$ is the number of times feature $i $ appears in a sample of class in the training set $T$, and $N_{y} = \sum^{n}_{i=1}{N_{yi}}$ is the total count of all features for class $y$.

- The smoothing priors $\alpha \gt 0$ accounts for features not present in the learning samples and **prevents zero probabilities** in further computations. Setting $\alpha = 1$ is called Laplace smoothing, while $\alpha \lt 1$ is called Lidstone smoothing.


In [14]:
'''
attr = dataframe consisting vectorized attributes
labels = dataframe consisting of numerical labels
alpha(float) = smoothing parameter
prior = list containing prior probabilities for each class i  [1,2,...m] classes


returns: All possible condition probability possible 
'''
def TrainNaiveBayes(attr, labels,alpha,priors):
    x,y = attr, labels.to_numpy()
    m, n,k = len(np.unique(y)), len(x[0]), len(x)
    CondProb={}
    for j in range(m):
        CondProb[j] = []
        att= np.array([x[i] for i in range(k) if y[i] == j])
        for i in range(n):
            val, cnt=np.unique(att[:,i],return_counts=True)
            CondProb[j].append({val[i]:(cnt[i]+alpha)/(np.sum(cnt)+alpha*n) for i in range(len(val))})
    return CondProb    

## Vectorizing Training Sample

- Use the Helper code above to vectorize for training samples
- Don't overthink it, its very easy to do

In [15]:
def Vectorizer(X):
    v =CountVectorizer()
    x= v.fit_transform(X)
    return v, (x.toarray())
    
vect, VecX_train= Vectorizer(X_train)
VecX_train.shape

(4179, 7996)

## Calculate Priors and Estimate Model's performance on Training Sample

- Calculate priors based on Training Sample using your NB implementation
- Evaluate your model's performance on Training Data ($\alpha = 0$)

In [16]:
"""
Calculting Prior Probability for each class
"""
y= y_train.to_numpy()
alpha = 0
m=len(np.unique(y)) # number of class
classes, counts = np.unique(y, return_counts=True)
Prior =[counts[i]/len(y_train) for i in range(m)]
CondProb =TrainNaiveBayes(VecX_train,y_train,alpha,Prior)


In [17]:
'''
Testing the model on the trainData
'''
'''
X, y: data to be tested
Prior: prior probabilities of each classes
Cond: Condition Probabilities of each attribute
alpha: Smoothing Parameter
K: length of the training dataset
'''
def Predict(X,y, Prior, Cond,alpha,K):
    y=y.to_numpy()
    m = len(np.unique(y))
    n = len(X[0])
    acc=0
    y_pred = np.zeros(len(y))
    for k,x in (enumerate(X)):
        p = np.ones(m)
        for j in range(m):
            for i in range(n):
                if x[i] in Cond[j][i].keys():
                    p[j]*=Cond[j][i][x[i]]
                else:
                    p[j]*=alpha/(alpha*n + Prior[j]*K)
                        
            p[j]*=Prior[j]
        pred= np.argmax(p)
        if(pred == y[k]):
            acc+=1
        y_pred[k]=pred
    return y_pred, acc/len(y)
y_pred,train_acc =Predict(VecX_train,y_train, Prior,CondProb,alpha,len(y_train))
print(f'accuracy on train dataset is {train_acc*100}%')

accuracy on train dataset is 99.9760708303422%


## Vectorizing Test Sample

- Use the Training Sample vocabulary to create word count matrix for test samples
- This is also shown in the Helper code

In [18]:
VecX_test = vect.transform(X_test).toarray()
VecX_test.shape

(1393, 7996)

## Estimate Model's performance on Test Sample

- Evaluate your model's performance on Test Sample, using the Training Priors ($\alpha = 0$)

In [19]:
y_test_pred,test_acc =Predict(VecX_test,y_test, Prior,CondProb,alpha,len(y_train))
print(f'accuracy on test dataset is {test_acc*100}%')

accuracy on test dataset is 95.40559942569993%


## Select Smoothing Priors

- Refactor your code to incorporate smoothing priors, select $\alpha = 0$ for the previous estimates / sub-questions
- Compare the performance with different values of $\alpha \gt 0$ as smoothing priors to take care of zero probabilities
- You can display a Plot or Table to show the comparison.

In [20]:
A = [0,1e-9,1e-7,1e-5,0.001, 0.1,1]
for alpha in A:
    CondProb =TrainNaiveBayes(VecX_train,y_train,alpha,Prior)
    print(f'alpha: {alpha}')
    y_smooth_pred,smooth_acc =Predict(VecX_train,y_train, Prior,CondProb,alpha,len(y_train))
    print(f'accuracy on train dataset is {smooth_acc*100}%')
    y_smooth_pred,smooth_acc =Predict(VecX_test,y_test, Prior,CondProb,alpha,len(y_train))
    print(f'accuracy on test dataset is {smooth_acc*100}%')

alpha: 0
accuracy on train dataset is 99.9760708303422%
accuracy on test dataset is 95.40559942569993%
alpha: 1e-09
accuracy on train dataset is 99.9760708303422%
accuracy on test dataset is 98.77961234745155%
alpha: 1e-07
accuracy on train dataset is 99.9760708303422%
accuracy on test dataset is 98.77961234745155%
alpha: 1e-05
accuracy on train dataset is 99.95214166068438%
accuracy on test dataset is 98.7078248384781%
alpha: 0.001
accuracy on train dataset is 91.38549892318737%
accuracy on test dataset is 90.23689877961235%
alpha: 0.1
accuracy on train dataset is 86.4321608040201%
accuracy on test dataset is 87.07824838478105%
alpha: 1
accuracy on train dataset is 86.4321608040201%
accuracy on test dataset is 87.07824838478105%


## Comparison with Sci-kit Learn Implementation

- Use sci-kit learn's `sklearn.naive_bayes.MultinomialNB` model to compare your implementation's performance
- (Optional) try other classifiers from `sklearn.naive_bayes` and see if you can make them work`

In [21]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
for alpha in A:
    print(f'alpha: {alpha}')
    clf = MultinomialNB(alpha=alpha, force_alpha=True)
    clf.fit(VecX_train, y_train)
    ytrain_pred=clf.predict(VecX_train)
    ytest_pred=clf.predict(VecX_test)
    print(f'Accuracy on train is {accuracy_score(y_train,ytrain_pred)}')
    print(f'Accuracy on test is {accuracy_score(y_test,ytest_pred)}')


alpha: 0


/home/user/anaconda3/envs/ads/lib/python3.10/site-packages/sklearn/naive_bayes.py:905: RuntimeWarning: divide by zero encountered in log
  self.feature_log_prob_ = np.log(smoothed_fc) - np.log(
/home/user/anaconda3/envs/ads/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/user/anaconda3/envs/ads/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


Accuracy on train is 0.864321608040201
Accuracy on test is 0.8707824838478104
alpha: 1e-09
Accuracy on train is 0.9971284996410624
Accuracy on test is 0.9820531227566404
alpha: 1e-07
Accuracy on train is 0.9971284996410624
Accuracy on test is 0.9820531227566404
alpha: 1e-05
Accuracy on train is 0.9971284996410624
Accuracy on test is 0.9827709978463748
alpha: 0.001
Accuracy on train is 0.9971284996410624
Accuracy on test is 0.9827709978463748
alpha: 0.1
Accuracy on train is 0.9956927494615937
Accuracy on test is 0.9834888729361091
alpha: 1
Accuracy on train is 0.994256999282125
Accuracy on test is 0.9827709978463748
